In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
pip install torch torchvision torchaudio


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as T
import gradio as gr
import matplotlib.pyplot as plt

import sys
sys.path.append('/content/drive/MyDrive/DeepLabV3Plus-Pytorch')

# Load the model
from network.modeling import deeplabv3plus_resnet50
model = deeplabv3plus_resnet50(num_classes=3)
checkpoint_path = '/content/drive/MyDrive/DeepLabV3Plus-Pytorch/checkpoints/best_deeplabv3plus_resnet50_voc_os16_experiment1_lr=0.01.pth'
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state'])
model.eval()

# Preprocessing transforms
transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Segmentation function
def segment_image(input_image):
    try:
        # Convert Gradio's NumPy array input to a PIL image
        image = Image.fromarray(input_image).convert('RGB')
        original_size = image.size

        # Resize and preprocess the image
        image = image.resize((513, 513))
        x = transform(image).unsqueeze(0)

        with torch.no_grad():
            out = model(x)
            pred = out.squeeze(0).argmax(0).cpu().numpy()

        # Generate output mask
        output = np.zeros((513, 513, 3), dtype=np.uint8)
        output[pred == 1] = [0, 0, 255]  # Swimming pools in blue
        output[pred == 2] = [0, 255, 0]  # Tennis courts in green

        # Resize output back to the original size
        output_img = Image.fromarray(output).resize(original_size, Image.NEAREST)
        return np.array(output_img)

    except Exception as e:
        print("Error during segmentation:", str(e))
        # Return a blank image in case of an error
        return np.zeros_like(input_image, dtype=np.uint8)

# Gradio Interface
interface = gr.Interface(
    fn=segment_image,
    inputs=gr.Image(),
    outputs=gr.Image(),
    title="Swimming Pool and Tennis Court Detector",
    description="Upload an image to detect swimming pools (blue) and tennis courts (green)."
)

interface.launch(share=True)


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 409MB/s]
<ipython-input-4-55b1caccd7ef>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://476a49040acd7fcb2e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
